In [1]:
import geopandas as gpd
import plotly
import matplotlib
import pandas as pd
import numpy as np

In [2]:
tazs = gpd.read_file('C:\\Users\phoebe.AD\Downloads\IntersectionDensity\\TAZs_NAD83.shp') #no water

edges = gpd.read_file('C:\\Users\phoebe.AD\Downloads\IntersectionDensity\edges.shp')

nodes = gpd.read_file('C:\\Users\phoebe.AD\Downloads\IntersectionDensity\\nodes.shp')

In [3]:
tazs = tazs.to_crs(26986)
edges = edges.to_crs(26986)
nodes = nodes.to_crs(26986)

In [4]:
nos_edges = edges[~edges['highway'].str.contains('service')]

In [14]:
sub_nodes = nodes[(nodes['osmid'].isin(nos_edges['u']) | nodes['osmid'].isin(nos_edges['u'])) & (nodes['street_cou'] > 2)]

In [15]:
#buffer!
tazs['geometry']=tazs.buffer(100)

In [16]:
#get counts of points in each polygon (buffered tazs)
spjoin = gpd.sjoin(tazs, sub_nodes)

In [17]:
#get count of points per polygon
pperp_ped = spjoin.groupby(['id']).count().reset_index()

In [18]:
#get a count - but has to be a column that has been 100% filled in the original data
pperp_ped['intersection_count'] =  pperp_ped['OBJECTID']
taz_ints_ped = pperp_ped[['id','intersection_count']]

In [25]:
#get square miles
tazs["area"] = tazs.area
tazs['Area_SqMi'] =  tazs['area']*0.00000038610

In [26]:
#get density
taz_area = tazs[['id','Area_SqMi']]
taz_den_ped = taz_ints_ped.merge(taz_area, how = 'outer', on='id')

In [27]:
taz_den_ped['intersection_density'] = taz_den_ped['intersection_count']/taz_den_ped['Area_SqMi']

In [28]:
taz_den_ped.head()

,id,intersection_count,Area_SqMi,intersection_density
0,1,255.0,0.257783,989.202791
1,2,248.0,0.149527,1658.568164
2,3,436.0,0.165858,2628.760887
3,4,340.0,0.190535,1784.449664
4,5,392.0,0.202938,1931.627485


In [29]:
taz_den_ped.to_csv('J:\Shared drives\TMD_TSA\Data\\Non_Motorized_Networks\Intersections_per_TAZ\ped_intersection_density_dec20.csv')